# Выжившие на Титанике

Даны 2 набора данных: трейн и тест о пассажирах титаника. 

В трейне известна летальность. Нужно предсказать, сколько умерших в тесте. Метрика accuracy.

## Импорты

Работать будем с пандасом, бустингами, лесом и логрегом, т.к. задача бинарной классификации по типу "да/нет"

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score


from sklearn.model_selection import train_test_split


## Предобработка

Посмотрим, что там у нас за данные:

In [2]:
train = pd.read_csv('../input/titanic/train.csv')
train.head()

In [3]:
test = pd.read_csv('../input/titanic/test.csv')
test.head()

Таблица соотношений:

In [4]:
train.describe()

Самому старому старику 80 лет. Интересно, он выживет?) 512 максимальное расстояние до выхода (если я правильно понимаб признак Fare) шагов или метров... В принципе, если в панике быстро бежать, то шаг - это как раз метр)

**Распределения ещё посмотрим на паре графиков:**

In [5]:
train.boxplot(figsize=(10,6))
plt.show()

In [6]:
train.hist()
plt.show()

In [7]:
test.hist()
plt.show()

Ситуация, конечно, плохая с этим краблём получилась, труппов больше, чем выживших примерно в 2 раза, но думаю, что для обучения этого хватит и масштабирование признаков не понадобится.

Больше всего ехало бедняков 3-м классом, молодых, возрастом 20-30 лет в среднем. Вообще возраст стандартно распределён, на корабле,как и в жизни, так сказать )

Посмотрим на билеты:

- Некоторые билеты повторяются, оставим столбец, как признак

In [8]:
train['Ticket'].value_counts()

Там в кабинах сразу наны в таблице выдны, изучим столбец:

In [9]:
train['Cabin'].value_counts()

Лучше посмотреть на это разнообразие через уникальные значения:

In [10]:
train['Cabin'].unique()

Похоже на классовое распределение кабин, можно применить классификацию: A, B, C, D, E, G, F и T.

Но для начала уберём пропуски, заменив их на другую классификацию (может это какие нибудь кабины для нищих или что-то в этом роде.

In [11]:
train['Cabin'] = train['Cabin'].fillna('other')

In [12]:
train.head()

Добавим признак:

In [13]:
def cab_satus(row):
    if 'A' in row['Cabin']:
        return 'A'
    elif 'B' in row['Cabin']:
        return 'B'
    elif 'C' in row['Cabin']:
        return 'C'
    elif 'D' in row['Cabin']:
        return 'D'
    elif 'E' in row['Cabin']:
        return 'E'
    elif 'F' in row['Cabin']:
        return 'F'
    elif 'G' in row['Cabin']:
        return 'G'
    elif 'T' in row['Cabin']:
        return 'T'
    else:
        return 'other'
    
train['Cabin_stat'] = train.apply(cab_satus, axis=1)

In [14]:
train['Cabin_stat'].unique()

И в тестовую сразу добавим, заодно наны из её кабин уберём:

In [15]:
test['Cabin'].unique()

In [16]:
test['Cabin'] = test['Cabin'].fillna('other')

In [17]:

test['Cabin_stat'] = test.apply(cab_satus, axis=1)
test['Cabin_stat'].unique()

Посмотрим, чё там ещё с пропусками:

In [18]:
train.isna().sum()

In [19]:
test.isna().sum()

In [20]:
train['Age'].unique()

Возраст, мы помним, распределён нормально, поэтому пропуски заполним средним значением.

In [21]:
mean_age = train['Age'].mean()

In [22]:
train['Age'] = train['Age'].fillna(mean_age)
train.isna().sum()

In [23]:
train['Age'].unique()

In [24]:
test['Age'] = test['Age'].fillna(mean_age)
test.isna().sum()

Ещё возраст лучше округлить

In [25]:
train['Age'] = train['Age'].astype('uint8')
test['Age'] = test['Age'].astype('uint8')

Не особо вникла, что это за признак, но 1 и 2 пропуска можно просто удалить

In [26]:
train['Embarked'].value_counts()

In [27]:
train = train.dropna()
train.isna().sum()

Видимо удалённость от выхода:

In [28]:
test['Fare'].value_counts()

In [29]:
test = test.dropna()
test.isna().sum()

Вынесем уникальные номера пассажиров в индексы таблиц:

In [30]:
train.set_index('PassengerId', inplace=True)
train.head(3)

In [31]:
test.set_index('PassengerId', inplace=True)
test.head(3)

In [32]:
train['Ticket'].unique()

Думаю, что с таким количеством уникальных значений билеты, как и имена пассажиров в обучении модели участвовать не будут.

**Посмотрим-ка ещё раз на нашу "причёсанную" таблицу:**

In [33]:
train.info()
train.head(3)

In [34]:
test.info()
test.head(3)

Отлично :) Выглядит хорошо.

## Разбивка на цели/признаки

Создавая признаки избавимся заодно от "груза" по типу уникальных значений имён пользователей и билетов, хватит и индексов, чтоб потом всё вернуть на место.

In [35]:
target_train = train['Survived'] #конечно же цель - определить выживание
features_train = train.drop(['Name', 'Ticket', 'Survived'], axis=1)

In [36]:
features_test = test.drop(['Name', 'Ticket'], axis=1)

In [37]:
features_train.head(3) #проверка

Разобьём трейн на трейн и валид, что б понять, насколько хороши наши модели.

In [38]:
 features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, test_size=0.3, random_state=12345)

Обычно под валид выделяют 30%/

In [39]:
"Соотношение тренировочной к тестовой выборке:", target_train.shape, target_valid.shape

## Анализ моделей

### Логистическая регрессия

Придётся сделать прямое кодирование, она буквы не читает.

In [40]:
features_train_ohe = pd.get_dummies(features_train.drop(['Cabin', 'Cabin_stat'], axis=1), drop_first=True)

features_train_ohe.info()
features_train_ohe.head()

In [41]:
features_valid_ohe = pd.get_dummies(features_valid.drop(['Cabin', 'Cabin_stat'], axis=1), drop_first=True)

features_valid_ohe.head(3)

Тест тоже заодно изменим, вдруг логрег станет лучшей моделью:

In [42]:
features_test_ohe = pd.get_dummies(features_test.drop(['Cabin', 'Cabin_stat'], axis=1), drop_first=True)

features_test_ohe.head(3)

Пришлось удалить некоторые признаки, т.к. при прямом кодировании столбцов получается неодинаковое количество и модели не работают.

**Создадим модель:**

In [43]:
model_log = LogisticRegression(random_state=12345, solver='liblinear')
model_log.fit(features_train_ohe, target_train)
predicted_log_valid = model_log.predict(features_valid_ohe)

print("Предсказание несбалансированной модели:", model_log.score(features_valid_ohe, target_valid))
print("Accuracy:", accuracy_score(target_valid, predicted_log_valid))
print("F1:", f1_score(target_valid, predicted_log_valid))

### Модель леса

In [44]:
best_model_forest = None
best_result_forest = 0
best_predict_forest = None

for est in range(1, 50):
    for depth in range(1, 12):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) # обучим модель с заданным количеством деревьев
        model_forest.fit(features_train_ohe, target_train) # обучим модель на тренировочной выборке
        result_forest = model_forest.score(features_valid_ohe, target_valid) # посчитаем качество модели на валидационной выборке
        predict_forest = model_forest.predict(features_valid_ohe)

        if result_forest > best_result_forest:
            best_model_forest = model_forest # сохраним наилучшую модель
            best_result_forest = result_forest #  сохраним наилучшее значение метрики accuracy на валидационных данных
            best_predict_forest = predict_forest

print("Accuracy наилучшей модели случайного леса на валидационной выборке:", best_result_forest)
print("F1:", f1_score(target_valid, best_predict_forest))

Кстати тут мы не используем гридсёрч, потому что он меня бесит своей медлительностью. Лучше использовать библиотеку рандомного подбора `Optuna`, но её мы тоже использовать не будем, т.к. это кагловский конкурс, а с оптуной кагл не дружит. Пусть будет обычный цикл.

### CatBoost

Для этого зверя нужно определить столбцы-категории, высчитать индексы этих столбцов. Капец как неудобно, но что поделать.

In [45]:
features_train.info()

Вот слева от столбцов их индексы.

In [46]:
cat_features_list = [1, 6, 7, 8] #это мы передадим в параметры модели

In [47]:
model_cat = CatBoostClassifier(iterations=70, cat_features=cat_features_list, random_seed=12345)

Посчитаем время и важность признаков заодно:

In [48]:
%%timeit
model_cat.fit(features_train, target_train, verbose=10)

In [49]:
predicted_valid_cat = model_cat.predict(features_valid)
print("accuracy модели catboost:", accuracy_score(target_valid, predicted_valid_cat))
print("f1 модели catboost:", f1_score(target_valid, predicted_valid_cat))

In [50]:
start_time = time.time()
importances_cat_valid = model_cat.feature_importances_
std = np.std([
    model_cat.feature_importances_], axis=0)
elapsed_time = time.time() - start_time

print(f"Затраченное время для вычисления важности: "
      f"{elapsed_time:.3f} секунд")

cat_importances_valid = pd.Series(importances_cat_valid, index=features_valid.columns)

fig, ax = plt.subplots(figsize=(6, 6))
cat_importances_valid.plot.bar(yerr=std, ax=ax)
ax.set_title("Значения функций с использованием MDI для CatBoost")
ax.set_ylabel("Среднее снижение содержания примесей")
fig.tight_layout()

### LightGBM

Нужно определить категориальные столбцы для этой модели

In [51]:
features_train[
    ['Sex', 
     'Cabin', 
     'Embarked',  
     'Cabin_stat']
] = features_train[
    ['Sex', 
     'Cabin', 
     'Embarked',  
     'Cabin_stat']
].astype('category')

features_valid[
    ['Sex', 
     'Cabin', 
     'Embarked',  
     'Cabin_stat']
] = features_valid[
    ['Sex', 
     'Cabin', 
     'Embarked',  
     'Cabin_stat']
].astype('category')

features_test[
    ['Sex', 
     'Cabin', 
     'Embarked',  
     'Cabin_stat']
] = features_test[
    ['Sex', 
     'Cabin', 
     'Embarked',  
     'Cabin_stat']
].astype('category')

In [52]:
model_gbm = LGBMClassifier(random_state=12345, class_weight='balanced')

In [53]:
%%timeit
model_gbm.fit(features_train, target_train, verbose=10)

In [54]:
predicted_valid_gbm = model_gbm.predict(features_valid)
print("accuracy модели LGBM:", accuracy_score(target_valid, predicted_valid_gbm))
print("f1 модели LGBM:", f1_score(target_valid, predicted_valid_gbm))

### Вывод

В общем лучше всех моделей справилась модель случайного леса. Её и будем использовать. Градиентный бустинг проиграл :))

## Получение предсказаний на тестовых признаках!

Хорошим тоном будет объединение выборок обратно в одну большую тестовую.

In [55]:
features_train_valid = pd.concat([features_train_ohe, features_valid_ohe])
target_train_valid = pd.concat([target_train, target_valid])

Обучим модель леса на случайной выборке и получим предсказания.

In [56]:
best_model_forest.fit(features_train_valid, target_train_valid)
predicted_test = best_model_forest.predict(features_test_ohe)


Теперь перенесём в сериес предсказания

In [57]:
predict = pd.Series(predicted_test)


Вернём индекс в столбец таблицы (так нужно по условию проекта)

In [58]:
test = test.rename_axis('PassengerId').reset_index()
test

вынесем нумеровку пассажиров как сериес (мы будем делать итоговую таблицу).

In [59]:
passenger = test['PassengerId']
passenger

Подготовим словарик для нашей итоговой таблицы:

In [60]:

data = {
    'PassengerId': passenger, 
    'Survived': predict
}

А теперь создадим, наконец-то, таблицу

In [61]:
df = pd.DataFrame(data=data)
df

И выведем её наружу из тетради)

In [62]:

df.to_csv(r'predict.csv')